In [10]:
import random
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import copy
import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
# Network (now predicts only the median)
class NextStateMedianNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(NextStateMedianNetwork, self).__init__()
        self.layer1 = nn.Linear(state_dim + action_dim, 256)
        self.layer2 = nn.Linear(256, 256)
        self.layer3 = nn.Linear(256, state_dim)  # Single output per state dim

    def forward(self, state, action):
        if len(state.shape) == 1:
            x = torch.cat((action, state))
        else:
            x = torch.cat((action, state), dim=1) # .unsqueeze(1)
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return self.layer3(x)

def quantile_loss_median(predicted, target):
    error = target - predicted
    quantile = torch.tensor(0.5)
    loss = torch.max(
        quantile * error,
        (quantile - 1) * error
    )
    return loss.mean()

def mse_loss(predicted, target):
    error = target - predicted
    return error.pow(2).mean()
    

Test where I removed the quadratic part, need to put the threshold back to 1 to back to standard.

In [12]:
class NextStateQuantileNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, num_quantiles):
        super(NextStateQuantileNetwork, self).__init__()
        self.num_quantiles = num_quantiles

        # Input layer (state + action concatenation)
        self.layer1 = nn.Linear(state_dim + action_dim, 256)
        self.layer2 = nn.Linear(256, 256)
        self.layer3 = nn.Linear(256, state_dim * num_quantiles)  # Output quantiles for each state dimension
        # self.layer3 = torch.tanh(256, state_dim * num_quantiles)  # Output quantiles for each state dimension

    def forward(self, state, action):
        # Concatenate state and action
        # x = torch.cat((action, state))
        # print("action ", action, "\n")
        
        # print("state ", state, "\n")
        # print("state.shape ", state.shape, "\n")
        # print("action ", action, "\n")
        # print("action.shape ", action.shape, "\n")
        
        if len(state.shape) == 1:
            x = torch.cat((action, state))
        else:
            x = torch.cat((action, state), dim=1) # .unsqueeze(1)
            
        # print("x ", x, "\n")
        # print("x.shape ", x.shape, "\n")
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x.view(-1, self.num_quantiles, state.size(-1))


def quantile_huber_loss(predicted, target, quantiles, batch_size=32):
    """
    Calculate Quantile Huber Loss.
    :param predicted: Predicted quantiles, shape (batch_size, state_dim, num_quantiles)
    :param target: Target next state, shape (batch_size, state_dim)
    :param quantiles: Quantiles (e.g., [0.1, 0.3, 0.7, 0.9]), shape (num_quantiles,)
    """
    
    # print("target.shape ", target.shape, "\n")
    # print("target ", target, "\n")
    # target = target.unsqueeze(-1)  # Shape: (batch_size, state_dim, 1)
    target = target.unsqueeze(1).repeat(1,len(quantiles),1)
    # print("target ", target, "\n")
    # print("predicted ", predicted, "\n")
    # print("target.shape ", target.shape, "\n")
    # print("predicted.shape ", predicted.shape, "\n")
    
    error = target - predicted  # Shape: (batch_size, state_dim, num_quantiles)
    
    # print("error ", error, "\n")
    quantiles = quantiles.view(1, -1, 1)
    quantiles = quantiles.repeat(batch_size, 1, target.shape[-1])  # Shape: [3, 4, 2]
    # quantiles = quantiles.repeat(batch_size, target.shape[-1], 1) 
    # quantiles = quantiles.transpose(1, 2)  # Shape: [3, 2, 4]
    # print("quantiles ", quantiles, "\n")
    
    # # Make delta adaptive by scaling it based on quantiles
    # delta = 1.0
    # adaptive_delta = delta * (1.0 + torch.abs(quantiles - 0.5))  # Give more tolerance to extreme quantiles
    
    # # print("quantiles.shape ", quantiles.shape, "\n")
    # # Calculate loss
    # huber_loss = torch.where(
    #     error.abs() <= 0.0, # 1.0
    #     0.5 * error.pow(2),
    #     error.abs() - 0.5
    # )

    # # huber_loss = error.abs()-0.5  # Simple L1 loss
    
    # # print("huber_loss ", huber_loss, "\n")
    # # print("huber_loss.shape ", huber_loss.shape, "\n")
    
    # # Quantile loss computation
    # quantile_loss = (quantiles - (error < 0).float()).abs() * huber_loss

    # Standard Quantile Loss (Pinball Loss)
    quantile_loss = torch.max(
        quantiles * error,
        (quantiles - 1) * error
    )

    return quantile_loss.mean()

    # Quantile loss computation
    # quantile_loss = torch.abs(quantiles - (error.detach() < 0).float()) * huber_loss
    # return quantile_loss.sum(dim=1).mean()

# def quantile_huber_loss(predicted, target, quantiles, batch_size=32):
#     """
#     Calculate Quantile Huber Loss.
#     :param predicted: Predicted quantiles, shape (batch_size, state_dim, num_quantiles)
#     :param target: Target next state, shape (batch_size, state_dim)
#     :param quantiles: Quantiles (e.g., [0.1, 0.3, 0.7, 0.9]), shape (num_quantiles,)
#     """
    
#     # print("target.shape ", target.shape, "\n")
#     # print("target ", target, "\n")
#     # target = target.unsqueeze(-1)  # Shape: (batch_size, state_dim, 1)
#     target = target.unsqueeze(1).repeat(1,len(quantiles),1)
#     print("target ", target, "\n")
#     print("predicted ", predicted, "\n")
#     print("target.shape ", target.shape, "\n")
#     print("predicted.shape ", predicted.shape, "\n")
    
#     error = target - predicted  # Shape: (batch_size, state_dim, num_quantiles)
    
#     # print("error ", error, "\n")
#     quantiles = quantiles.view(1, -1, 1)
#     quantiles = quantiles.repeat(batch_size, 1, target.shape[-1])  # Shape: [3, 4, 2]
#     # quantiles = quantiles.repeat(batch_size, target.shape[-1], 1) 
#     # quantiles = quantiles.transpose(1, 2)  # Shape: [3, 2, 4]
#     print("quantiles ", quantiles, "\n")
    
#     # print("quantiles.shape ", quantiles.shape, "\n")
#     # Calculate loss
#     huber_loss = torch.where(
#         error.abs() <= 1.0,
#         0.5 * error.pow(2),
#         error.abs() - 0.5
#     )
    
#     # print("huber_loss ", huber_loss, "\n")
#     # print("huber_loss.shape ", huber_loss.shape, "\n")
    
#     # Quantile loss computation
#     # quantile_loss = (quantiles - (error < 0).float()).abs() * huber_loss
#     # return quantile_loss.mean()

#     # Quantile loss computation
#     quantile_loss = torch.abs(quantiles - (error.detach() < 0).float()) * huber_loss
#     return quantile_loss.sum(dim=1).mean()


Graph of number of values below the different quantiles graph and calculation functions


In [13]:
def plot_nb_below_quantiles(nb_belowq1_list, nb_belowq2_list, nb_belowq3_list, nb_belowq4_list, nb_belowq5_list, nb_belowq6_list, nb_belowq7_list, nb_belowq8_list, nb_belowq9_list, nb_belowq10_list, variable):
    
    if variable == "theta":
        variable = r"$\theta$"

    plt.figure(1)
    # plt.plot(nb_belowq1_list, label=f'quantile1_{variable}')
    # plt.plot(nb_belowq2_list, label=f'quantile2_{variable}')
    # plt.plot(nb_belowq3_list, label=f'quantile3_{variable}')
    # plt.plot(nb_belowq4_list, label=f'quantile4_{variable}')
    # plt.plot(nb_belowq5_list, label=f'quantile5_{variable}')
    # plt.plot(nb_belowq6_list, label=f'quantile6_{variable}')
    # plt.plot(nb_belowq7_list, label=f'quantile7_{variable}')
    # plt.plot(nb_belowq8_list, label=f'quantile8_{variable}')
    # plt.plot(nb_belowq9_list, label=f'quantile9_{variable}')
    # plt.plot(nb_belowq10_list, label=f'quantile10_{variable}')
    plt.plot(nb_belowq1_list, label=f'Q1_{variable}')
    plt.plot(nb_belowq2_list, label=f'Q2_{variable}')
    plt.plot(nb_belowq3_list, label=f'Q3_{variable}')
    plt.plot(nb_belowq4_list, label=f'Q4_{variable}')
    plt.plot(nb_belowq5_list, label=f'Q5_{variable}')
    plt.plot(nb_belowq6_list, label=f'Q6_{variable}')
    plt.plot(nb_belowq7_list, label=f'Q7_{variable}')
    plt.plot(nb_belowq8_list, label=f'Q8_{variable}')
    plt.plot(nb_belowq9_list, label=f'Q9_{variable}')
    plt.plot(nb_belowq10_list, label=f'Q10_{variable}')
    plt.hlines(0.1, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile1'
    plt.hlines(0.2, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile2'
    plt.hlines(0.3, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile3'
    plt.hlines(0.4, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile4'
    plt.hlines(0.5, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile5'
    plt.hlines(0.6, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile6'
    plt.hlines(0.7, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile7'
    plt.hlines(0.8, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile8'
    plt.hlines(0.9, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile9'
    plt.hlines(1.0, 0, len(nb_belowq1_list), colors='r', linestyles='dashed') # , label='quantile10'
    plt.xlabel('Number of steps')
    plt.ylabel(f'Number of steps below quantile {variable}')
    plt.legend(bbox_to_anchor=(1.05, 1))
    
    

In [14]:
def nb_below_quantile(env, quantile0, quantile1, quantile2, quantile3, quantile4, quantile5, quantile6, quantile7, quantile8, quantile9, quantile10):
    
    nb_belowq1 = 0
    nb_belowq2 = 0
    nb_belowq3 = 0
    nb_belowq4 = 0
    nb_belowq5 = 0
    nb_belowq6 = 0
    nb_belowq7 = 0
    nb_belowq8 = 0
    nb_belowq9 = 0
    nb_belowq10 = 0
    
    nb_belowq1_list = [0]
    nb_belowq2_list = [0]
    nb_belowq3_list = [0]
    nb_belowq4_list = [0]
    nb_belowq5_list = [0]
    nb_belowq6_list = [0]
    nb_belowq7_list = [0]
    nb_belowq8_list = [0]
    nb_belowq9_list = [0]
    nb_belowq10_list = [0]
    
    
    for i in range(len(quantile0)):
        
        if env[i] < quantile1[i]:
            nb_belowq1 += 1
            
        if env[i] < quantile2[i]:
            nb_belowq2 += 1
            
        if env[i] < quantile3[i]:
            nb_belowq3 += 1
            
        if env[i] < quantile4[i]:
            nb_belowq4 += 1
            
        if env[i] < quantile5[i]:
            nb_belowq5 += 1
            
        if env[i] < quantile6[i]:
            nb_belowq6 += 1
            
        if env[i] < quantile7[i]:
            nb_belowq7 += 1
            
        if env[i] < quantile8[i]:
            nb_belowq8 += 1
            
        if env[i] < quantile9[i]:
            nb_belowq9 += 1
            
        if env[i] < quantile10[i]:
            nb_belowq10 += 1

        nb_belowq1_list.append(nb_belowq1/(i+1))
        nb_belowq2_list.append(nb_belowq2/(i+1))
        nb_belowq3_list.append(nb_belowq3/(i+1))
        nb_belowq4_list.append(nb_belowq4/(i+1))
        nb_belowq5_list.append(nb_belowq5/(i+1))
        nb_belowq6_list.append(nb_belowq6/(i+1))
        nb_belowq7_list.append(nb_belowq7/(i+1))
        nb_belowq8_list.append(nb_belowq8/(i+1))
        nb_belowq9_list.append(nb_belowq9/(i+1))
        nb_belowq10_list.append(nb_belowq10/(i+1))
        
        

    return nb_belowq1_list, nb_belowq2_list, nb_belowq3_list, nb_belowq4_list, nb_belowq5_list, nb_belowq6_list, nb_belowq7_list, nb_belowq8_list, nb_belowq9_list, nb_belowq10_list


## Acrobot

When taking a step in the env, the state variables are $\cos{(\theta_1)}, \sin{(\theta_1)}, \cos{(\theta_2)}, \sin{(\theta_2)}, \omega_1, \omega_2$.

When using env.state, we have $\theta_1, \theta_2, \omega_1, \omega_2$. This allows us to have more precision in the angles which is needed in the MPC cost function.

Trying different PF and pre-trained using rnd actions QRNN - Feb 5


- Train on randomly sampled actions

In [15]:
error_magnitudes_costheta1 = []
error_magnitudes_sintheta1 = []
error_magnitudes_costheta2 = []
error_magnitudes_sintheta2 = []
error_magnitudes_omega1 = []
error_magnitudes_omega2 = []

env_costheta1 = []
env_sintheta1 = []
env_costheta2 = []
env_sintheta2 = []
env_omega1 = []
env_omega2 = []

pred_costheta1 = []
pred_sintheta1 = []
pred_costheta2 = []
pred_sintheta2 = []
pred_omega1 = []
pred_omega2 = []

# pred_x_var = []
# pred_y_var = []
# pred_z_var = []

seed = 0

env = gym.make('Acrobot-v1')#.unwrapped
sim_env = gym.make('Acrobot-v1')#.unwrapped  # Additional simulation model for MPC
# max_episode_steps = 500
# env = gym.make('CartPole-v1').unwrapped
# sim_env = gym.make('CartPole-v1').unwrapped  # Additional simulation model for MPC
sim_env.reset(seed=seed)
env.reset(seed=seed)

# Hyperparameters
state_dim = env.observation_space.shape[0]#-2#-1 # Since we only care about angle and omega which are given using env.state
# action_dim = env.action_space.shape[0]  # For Pendulum, it's continuous
action_dim=1

# Initialize the Next-State Prediction Network
model = NextStateMedianNetwork(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

loss_func = quantile_loss_median

states_low = torch.tensor([-1, -1, -1, -1, -12.566371, -28.274334])
states_high = torch.tensor([1, 1, 1, 1, 12.566371, 28.274334])

batch_size = 32

# Experience replay buffer
replay_buffer = []
discrete = True
num_test_steps = 20000

state, _ = env.reset(seed=seed)
episode_reward = 0
actions_list = []
done = False
truncated = False

actions_taken = np.zeros(num_test_steps)
env.action_space.seed(seed)

for step in range(num_test_steps):
    # state = env.state
    action = env.action_space.sample()
    actions_taken[step] = action
    
    # Apply the first action from the optimized sequence
    next_state, reward, done, truncated, info = env.step(action)
    # next_state = env.state
    episode_reward += reward
    actions_list.append(action)
    
    if step >= 1:
        # test_env = env.copy()
        # test_action = test_env.action_space.sample()
        # test_next_state, _, _, _, _ = test_env.step(test_action)
        # test_next_state = test_next_state['observation'][:3]
        
        test_next_state = torch.tensor(next_state, dtype=torch.float32)
        # test_next_state = torch.clip(test_next_state, states_low, states_high)
        # print("test_next_state ", test_next_state, "\n")
        
        if discrete:
            test_action = torch.tensor(np.array([action]), dtype=torch.float32)#.unsqueeze(1)  # Example action
        else:
            test_action = torch.tensor(action, dtype=torch.float32)
            
        test_state = torch.tensor(state, dtype=torch.float32)#.reshape(1, -1)
        test_state = torch.clip(test_state, states_low, states_high)
        
        # Predict next state quantiles
        preds = model(test_state, test_action)  # Shape: (1, num_quantiles, state_dim)
        # lower_quantile = predicted_quantiles[:, 0, :]  # Shape: (1, state_dim)
        # mid_quantile = predicted_quantiles[:, int(num_quantiles/2), :].detach().numpy()  # Shape: (1, state_dim)
        # upper_quantile = predicted_quantiles[:, -1, :]  # Shape: (1, state_dim)
        # print("predicted_quantiles ", predicted_quantiles, "\n")
        # print("Lower Quantile:", lower_quantile, "\n")
        # print("Mid Quantile:", mid_quantile, "\n")
        # print("Upper Quantile:", upper_quantile, "\n")
        
        pred_costheta1 = np.append(pred_costheta1, preds[0].detach().numpy())
        pred_sintheta1 = np.append(pred_sintheta1, preds[1].detach().numpy())
        pred_costheta2 = np.append(pred_costheta2, preds[2].detach().numpy()) 
        pred_sintheta2 = np.append(pred_sintheta2, preds[3].detach().numpy())
        # pred_theta2 = np.append(pred_sintheta2, preds[3].detach().numpy())  
        pred_omega1 = np.append(pred_omega1, preds[4].detach().numpy())
        pred_omega2 = np.append(pred_omega2, preds[5].detach().numpy())
        
        deltacostheta1 = test_next_state[0] - preds[0]
        deltasintheta1 = test_next_state[1] - preds[1]
        deltacostheta2 = test_next_state[2] - preds[2]
        deltasintheta2 = test_next_state[3] - preds[3]
        deltaomega1 = test_next_state[4] - preds[4]
        deltaomega2 = test_next_state[5] - preds[5]

        # delta = env_test_state - mean_next_state.flatten()#.detach().numpy()
        # print("delta ", delta, "\n") # 6
        # print("delta_x ", deltax, "\n") # 7
        # error_magnitude = np.linalg.norm(delta)
        # error_magnitudes.append(error_magnitude)
        error_magnitudes_costheta1.append(np.abs(deltacostheta1.detach().numpy())) # .detach().numpy()
        error_magnitudes_sintheta1.append(np.abs(deltasintheta1.detach().numpy())) # .detach().numpy()
        error_magnitudes_costheta2.append(np.abs(deltacostheta2.detach().numpy())) # .detach().numpy()
        error_magnitudes_sintheta2.append(np.abs(deltasintheta2.detach().numpy())) # .detach().numpy()
        error_magnitudes_omega1.append(np.abs(deltaomega1.detach().numpy())) # .detach().numpy()
        error_magnitudes_omega2.append(np.abs(deltaomega2.detach().numpy())) # .detach().numpy()
        # env_next_states.append(env_test_state)
        env_costheta1.append(test_next_state[0])
        env_sintheta1.append(test_next_state[1])
        env_costheta2.append(test_next_state[2])
        env_sintheta2.append(test_next_state[3])
        env_omega1.append(test_next_state[4])
        env_omega2.append(test_next_state[5])
    
    # next_state = env.state.copy()
    # Store experience in replay buffer
    replay_buffer.append((state, np.array([action]), reward, next_state, done))
    if len(replay_buffer) < batch_size:
        pass
    else:
        batch = random.sample(replay_buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.float32)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)
        
        states = torch.clip(states, states_low, states_high)
        
        # Predict next state quantiles
        predicted_quantiles = model(states, actions)  # Shape: (batch_size, num_quantiles, state_dim)
        
        # Use next state as target (can be improved with target policy)
        target_quantiles = next_states
        
        # Compute the target quantiles (e.g., replicate next state across the quantile dimension)
        # target_quantiles = next_states.unsqueeze(-1).repeat(1, 1, num_quantiles)

        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)

        loss = loss_func(predicted_quantiles, target_quantiles)
        
        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)
        
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    state = next_state
    
    if done or truncated:
        state, _ = env.reset(seed=seed)
        episode_reward = 0
        actions_list = []
        done = False
        truncated = False
 

In [16]:
actions_taken


array([2., 1., 1., ..., 2., 2., 0.])

Plots for the acrobot env when using cos(theta1), sin(theta1), cos(theta2), sin(theta2), omega1, omega2


In [17]:
env.action_space.seed(seed)
env.action_space.sample()

2

In [18]:
env.action_space.sample()

1

In [19]:
print(np.mean(error_magnitudes_costheta1),
      np.mean(error_magnitudes_sintheta1),
      np.mean(error_magnitudes_costheta2),
      np.mean(error_magnitudes_sintheta2),
      np.mean(error_magnitudes_omega1),
      np.mean(error_magnitudes_omega2),)


0.014046903 0.012131164 0.015980767 0.015819404 0.021358531 0.031639658


In [20]:
print(np.mean(error_magnitudes_costheta1[-100:]),
      np.mean(error_magnitudes_sintheta1[-100:]),
      np.mean(error_magnitudes_costheta2[-100:]),
      np.mean(error_magnitudes_sintheta2[-100:]),
      np.mean(error_magnitudes_omega1[-100:]),
      np.mean(error_magnitudes_omega2[-100:]),)

0.011049359 0.010617394 0.014177004 0.014911825 0.019920189 0.030221928


In [21]:
print("All steps")
print(f"$\\cos(\\theta_1)$ comparison:",
np.mean(np.abs(np.array(pred_costheta1)-np.array(env_costheta1))))

print(f"$\\sin(\\theta_1)$ comparison:",
np.mean(np.abs(np.array(pred_sintheta1)-np.array(env_sintheta1))))

print(f"$\\cos(\\theta_2)$ comparison:",
np.mean(np.abs(np.array(pred_costheta2)-np.array(env_costheta2))))

print(f"$\\sin(\\theta_2)$ comparison:",
np.mean(np.abs(np.array(pred_sintheta2)-np.array(env_sintheta2))))

print(f"$(\\omega_1)$ comparison:",
np.mean(np.abs(np.array(pred_omega1)-np.array(env_omega1))))

print(f"$(\\omega_2)$ comparison:",
np.mean(np.abs(np.array(pred_omega2)-np.array(env_omega2))))




All steps
$\cos(\theta_1)$ comparison: 0.014046902393641541
$\sin(\theta_1)$ comparison: 0.012131163737253772
$\cos(\theta_2)$ comparison: 0.015980767850582948
$\sin(\theta_2)$ comparison: 0.015819404716732904
$(\omega_1)$ comparison: 0.02135853185516712
$(\omega_2)$ comparison: 0.031639661073600896


In [22]:
print("Amonsgt last 100 steps:")
print(f"$\\cos(\\theta_1)$ comparison:",
np.mean(np.abs(np.array(pred_costheta1[-100:])-np.array(env_costheta1[-100:]))))

print(f"$\\sin(\\theta_1)$ comparison:",
np.mean(np.abs(np.array(pred_sintheta1[-100:])-np.array(env_sintheta1[-100:]))))

print(f"$\\cos(\\theta_2)$ comparison:",
np.mean(np.abs(np.array(pred_costheta2[-100:])-np.array(env_costheta2[-100:]))))

print(f"$\\sin(\\theta_2)$ comparison:",
np.mean(np.abs(np.array(pred_sintheta2[-100:])-np.array(env_sintheta2[-100:]))))

print(f"$(\\omega_1)$ comparison:",
np.mean(np.abs(np.array(pred_omega1[-100:])-np.array(env_omega1[-100:]))))

print(f"$(\\omega_2)$ comparison:",
np.mean(np.abs(np.array(pred_omega2[-100:])-np.array(env_omega2[-100:]))))



Amonsgt last 100 steps:
$\cos(\theta_1)$ comparison: 0.011049357652664184
$\sin(\theta_1)$ comparison: 0.010617395592853428
$\cos(\theta_2)$ comparison: 0.014177004601806402
$\sin(\theta_2)$ comparison: 0.01491182418394601
$(\omega_1)$ comparison: 0.019920188840478657
$(\omega_2)$ comparison: 0.030221926495432854


In [23]:
print("Amonsgt last 2000 steps/Without first 18000 steps:")
print(f"$\\cos(\\theta_1)$ comparison:",
np.mean(np.abs(np.array(pred_costheta1[18000:])-np.array(env_costheta1[18000:]))))

print(f"$\\sin(\\theta_1)$ comparison:",
np.mean(np.abs(np.array(pred_sintheta1[18000:])-np.array(env_sintheta1[18000:]))))

print(f"$\\cos(\\theta_2)$ comparison:",
np.mean(np.abs(np.array(pred_costheta2[18000:])-np.array(env_costheta2[18000:]))))

print(f"$\\sin(\\theta_2)$ comparison:",
np.mean(np.abs(np.array(pred_sintheta2[18000:])-np.array(env_sintheta2[18000:]))))

print(f"$(\\omega_1)$ comparison:",
np.mean(np.abs(np.array(pred_omega1[18000:])-np.array(env_omega1[18000:]))))

print(f"$(\\omega_2)$ comparison:",
np.mean(np.abs(np.array(pred_omega2[18000:])-np.array(env_omega2[18000:]))))



Amonsgt last 2000 steps/Without first 18000 steps:
$\cos(\theta_1)$ comparison: 0.009719800132640328
$\sin(\theta_1)$ comparison: 0.009439033125026948
$\cos(\theta_2)$ comparison: 0.01079608149642615
$\sin(\theta_2)$ comparison: 0.011422194447918721
$(\omega_1)$ comparison: 0.014890437378497302
$(\omega_2)$ comparison: 0.022839846410075776


In [24]:
# plt.figure(1)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_costheta1)
# plt.ylabel(f'Error magnitudes $\\cos(\\theta_1)$')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_1_{timestamp}.png')
# plt.yscale('log')

# plt.figure(2)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_sintheta1)
# plt.ylabel(f'Error magnitudes $\\sin(\\theta_1)$')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_1_{timestamp}.png')
# plt.yscale('log')

# plt.figure(3)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_costheta2)
# plt.ylabel(f'Error magnitudes $\\cos(\\theta_1)$')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_2_{timestamp}.png')
# plt.yscale('log')

# plt.figure(4)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_sintheta2)
# plt.ylabel(f'Error magnitudes $\\sin(\\theta_2)$')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_2_{timestamp}.png')
# plt.yscale('log')

# plt.figure(5)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_omega1)
# plt.ylabel(f'Error magnitudes $\omega_1$')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_1_{timestamp}.png')
# plt.yscale('log')

# plt.figure(6)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_omega2)
# plt.ylabel(f'Error magnitudes $\omega_2$')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_2_{timestamp}.png')
# plt.yscale('log')

# plt.figure(7)
# plt.xlabel('Nb of steps')
# plt.ylabel(f'$\\cos(\\theta_1)$ comparison between env and QRNN')
# plt.plot(env_costheta1, label='env_next_states')
# plt.plot(pred_costheta1, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_theta1)), pred_theta1 - pred_theta1_bottom_1std, pred_theta1 + pred_theta1_top_2std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_theta1)), pred_theta1 - pred_theta1_bottom_2std, pred_theta1 + pred_theta1_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\angles_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(8)
# plt.xlabel('Nb of steps')
# plt.ylabel(f'$\\sin(\\theta_1)$ comparison between env and QRNN')
# plt.plot(env_sintheta1, label='env_next_states')
# plt.plot(pred_sintheta1, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_theta1)), pred_theta1 - pred_theta1_bottom_1std, pred_theta1 + pred_theta1_top_2std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_theta1)), pred_theta1 - pred_theta1_bottom_2std, pred_theta1 + pred_theta1_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\angles_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(9)
# plt.xlabel('Nb of steps')
# plt.ylabel(f'$\\cos(\\theta_2)$ comparison between env and QRNN')
# plt.plot(env_costheta2, label='env_next_states')
# plt.plot(pred_costheta2, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_theta2)), pred_theta2 - pred_theta2_bottom_1std, pred_theta2 + pred_theta2_top_2std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_theta2)), pred_theta2 - pred_theta2_bottom_2std, pred_theta2 + pred_theta2_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\omegas_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(10)
# plt.xlabel('Nb of steps')
# plt.ylabel(f'$\\sin(\\theta_2)$ comparison between env and QRNN')
# plt.plot(env_sintheta2, label='env_next_states')
# plt.plot(pred_sintheta2, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_theta2)), pred_theta2 - pred_theta2_bottom_1std, pred_theta2 + pred_theta2_top_2std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_theta2)), pred_theta2 - pred_theta2_bottom_2std, pred_theta2 + pred_theta2_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\omegas_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(11)
# plt.xlabel('Nb of steps')
# plt.ylabel(f'$\omega_1$ comparison between env and QRNN')
# plt.plot(env_omega1, label='env_next_states')
# plt.plot(pred_omega1, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_omega1)), pred_omega1 - pred_omega1_bottom_1std, pred_omega1 + pred_omega1_top_1std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_omega1)), pred_omega1 - pred_omega1_bottom_2std, pred_omega1 + pred_omega1_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\angles_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(12)
# plt.xlabel('Nb of steps')
# plt.ylabel(f'$\omega_2$ comparison between env and QRNN')
# plt.plot(env_omega2, label='env_next_states')
# plt.plot(pred_omega2, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_omega2)), pred_omega2 - pred_omega2_bottom_1std, pred_omega2 + pred_omega2_top_1std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_omega2)), pred_omega2 - pred_omega2_bottom_2std, pred_omega2 + pred_omega2_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\omegas_comparison_GP_env_{timestamp}.png')
# plt.legend()




## Cart pole

In [25]:
error_magnitudes_x = []
error_magnitudes_v = []
error_magnitudes_theta = []
error_magnitudes_omega = []

env_x = []
env_v = []
env_theta = []
env_omega = []

pred_x = []
pred_v = []
pred_theta = []
pred_omega = []

# pred_x_var = []
# pred_y_var = []
# pred_z_var = []

seed = 0

env = gym.make('CartPole-v1')#.unwrapped
sim_env = gym.make('CartPole-v1')#.unwrapped  # Additional simulation model for MPC
# max_episode_steps = 500
# env = gym.make('CartPole-v1').unwrapped
# sim_env = gym.make('CartPole-v1').unwrapped  # Additional simulation model for MPC
sim_env.reset(seed=seed)
env.reset(seed=seed)

# Hyperparameters
state_dim = env.observation_space.shape[0]#-2#-1 # Since we only care about angle and omega which are given using env.state
# action_dim = env.action_space.shape[0]  # For Pendulum, it's continuous
action_dim=1

# Initialize the Next-State Prediction Network
model = NextStateMedianNetwork(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

loss_func = quantile_loss_median

states_low = torch.tensor([-4.8, -torch.inf, -0.41887903, -torch.inf])
states_high = torch.tensor([4.8, torch.inf, 0.41887903, torch.inf])

batch_size = 32

# Experience replay buffer
replay_buffer = []
discrete = True
num_test_steps = 20000

state, _ = env.reset(seed=seed)
episode_reward = 0
actions_list = []
done = False
truncated = False

actions_taken = np.zeros(num_test_steps)
env.action_space.seed(seed)

for step in range(num_test_steps):
    # state = env.state
    action = env.action_space.sample()
    actions_taken[step] = action
    
    # Apply the first action from the optimized sequence
    next_state, reward, done, truncated, info = env.step(action)
    # next_state = env.state
    episode_reward += reward
    actions_list.append(action)
    
    if step >= 1:
        # test_env = env.copy()
        # test_action = test_env.action_space.sample()
        # test_next_state, _, _, _, _ = test_env.step(test_action)
        # test_next_state = test_next_state['observation'][:3]
        
        test_next_state = torch.tensor(next_state, dtype=torch.float32)
        # test_next_state = torch.clip(test_next_state, states_low, states_high)
        # print("test_next_state ", test_next_state, "\n")
        
        if discrete:
            test_action = torch.tensor(np.array([action]), dtype=torch.float32)#.unsqueeze(1)  # Example action
        else:
            test_action = torch.tensor(np.array([action]), dtype=torch.float32).unsqueeze(1)
            
        test_state = torch.tensor(state, dtype=torch.float32)#.reshape(1, -1)
        test_state = torch.clip(test_state, states_low, states_high)
        
        # Predict next state quantiles
        preds = model(test_state, test_action)  # Shape: (1, num_quantiles, state_dim)
        # lower_quantile = predicted_quantiles[:, 0, :]  # Shape: (1, state_dim)
        # mid_quantile = predicted_quantiles[:, int(num_quantiles/2), :].detach().numpy()  # Shape: (1, state_dim)
        # upper_quantile = predicted_quantiles[:, -1, :]  # Shape: (1, state_dim)
        # print("predicted_quantiles ", predicted_quantiles, "\n")
        # print("Lower Quantile:", lower_quantile, "\n")
        # print("Mid Quantile:", mid_quantile, "\n")
        # print("Upper Quantile:", upper_quantile, "\n")
        
        pred_x = np.append(pred_x, preds[0].detach().numpy())
        pred_v = np.append(pred_v, preds[1].detach().numpy())
        pred_theta = np.append(pred_theta, preds[2].detach().numpy()) 
        pred_omega = np.append(pred_omega, preds[3].detach().numpy())
        # # pred_theta2 = np.append(pred_sintheta2, preds[3].detach().numpy())  
        # pred_omega1 = np.append(pred_omega1, preds[4].detach().numpy())
        # pred_omega2 = np.append(pred_omega2, preds[5].detach().numpy())
        
        deltax = test_next_state[0] - preds[0]
        deltav = test_next_state[1] - preds[1]
        deltatheta = test_next_state[2] - preds[2]
        deltaomega = test_next_state[3] - preds[3]
        # deltaomega1 = test_next_state[4] - preds[4]
        # deltaomega2 = test_next_state[5] - preds[5]

        # delta = env_test_state - mean_next_state.flatten()#.detach().numpy()
        # print("delta ", delta, "\n") # 6
        # print("delta_x ", deltax, "\n") # 7
        # error_magnitude = np.linalg.norm(delta)
        # error_magnitudes.append(error_magnitude)
        error_magnitudes_x.append(np.abs(deltax.detach().numpy())) # .detach().numpy()
        error_magnitudes_v.append(np.abs(deltav.detach().numpy())) # .detach().numpy()
        error_magnitudes_theta.append(np.abs(deltatheta.detach().numpy())) # .detach().numpy()
        error_magnitudes_omega.append(np.abs(deltaomega.detach().numpy())) # .detach().numpy()
        # error_magnitudes_omega1.append(np.abs(deltaomega1.detach().numpy())) # .detach().numpy()
        # error_magnitudes_omega2.append(np.abs(deltaomega2.detach().numpy())) # .detach().numpy()
        # env_next_states.append(env_test_state)
        env_x.append(test_next_state[0])
        env_v.append(test_next_state[1])
        env_theta.append(test_next_state[2])
        env_omega.append(test_next_state[3])
        # env_omega1.append(test_next_state[4])
        # env_omega2.append(test_next_state[5])
    
    # next_state = env.state.copy()
    # Store experience in replay buffer
    if discrete:
        replay_buffer.append((state, np.array([action]), reward, next_state, done))
    else:
        replay_buffer.append((state, np.array(action), reward, next_state, done))
        
    if len(replay_buffer) < batch_size:
        pass
    else:
        batch = random.sample(replay_buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.float32)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)
        
        states = torch.clip(states, states_low, states_high)
        
        # Predict next state quantiles
        predicted_quantiles = model(states, actions)  # Shape: (batch_size, num_quantiles, state_dim)
        
        # Use next state as target (can be improved with target policy)
        target_quantiles = next_states
        
        # Compute the target quantiles (e.g., replicate next state across the quantile dimension)
        # target_quantiles = next_states.unsqueeze(-1).repeat(1, 1, num_quantiles)

        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)

        loss = loss_func(predicted_quantiles, target_quantiles)
        
        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)
        
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    state = next_state
    
    if done or truncated:
        state, _ = env.reset(seed=seed)
        episode_reward = 0
        actions_list = []
        done = False
        truncated = False
 

In [26]:
print("Last 100 steps")
print(f"x comparison:",
np.mean(np.abs(np.array(pred_x[-100:])-np.array(env_x[-100:]))))

print(f"v comparison:",
np.mean(np.abs(np.array(pred_v[-100:])-np.array(env_v[-100:]))))

print(f"$\\theta$ comparison:",
np.mean(np.abs(np.array(pred_theta[-100:])-np.array(env_theta[-100:]))))

print(f"$\\omega$ comparison:",
np.mean(np.abs(np.array(pred_omega[-100:])-np.array(env_omega[-100:]))))


Last 100 steps
x comparison: 0.0014790624752640724
v comparison: 0.0027963328547775746
$\theta$ comparison: 0.0023765639240446034
$\omega$ comparison: 0.003939865119755268


In [27]:
print("All steps")
print(f"x comparison:",
np.mean(np.abs(np.array(pred_x)-np.array(env_x))))

print(f"v comparison:",
np.mean(np.abs(np.array(pred_v)-np.array(env_v))))

print(f"$\\theta$ comparison:",
np.mean(np.abs(np.array(pred_theta)-np.array(env_theta))))

print(f"$\\omega$ comparison:",
np.mean(np.abs(np.array(pred_omega)-np.array(env_omega))))


All steps
x comparison: 0.0036918997990464794
v comparison: 0.006502414954312139
$\theta$ comparison: 0.0039861937003700465
$\omega$ comparison: 0.007583585796413758


Plots for cart pole env

In [28]:
# # Plot training results
# import matplotlib.pyplot as plt
# # Plot the rewards
# plt.figure(1)
# # plt.legend()
# # plt.grid()
# plt.plot(episode_reward_list)
# plt.xlabel('Nb of episodes')
# plt.ylabel('episode reward')
# # plt.title('MPC_Pendulum with Optimized Action Sequences')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\Graphs\\episode_rewards_withCPUforQRNN_{timestamp}.png')
# # # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\episode_rewards_withGPUforGP_{timestamp}.png')
# plt.yscale('log')

# plt.figure(2)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_x)
# plt.ylabel('error_magnitudes_x')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_1_{timestamp}.png')
# plt.yscale('log')

# plt.figure(3)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_v)
# plt.ylabel('error_magnitudes_v')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_2_{timestamp}.png')
# plt.yscale('log')

# plt.figure(4)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_theta)
# plt.ylabel('error_magnitudes_theta')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_1_{timestamp}.png')
# plt.yscale('log')

# plt.figure(5)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_omega)
# plt.ylabel('error_magnitudes_omega')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_2_{timestamp}.png')
# plt.yscale('log')

# plt.figure(6)
# plt.xlabel('Nb of steps')
# plt.ylabel('x comparison between env and QRNN')
# plt.plot(env_x, label='env_next_states')
# plt.plot(pred_x, label='pred_next_states')
# plt.fill_between(np.arange(len(pred_x)), pred_x - pred_x_bottom_1std, pred_x + pred_x_top_2std, alpha=0.5, label='1 std', color='green')
# plt.fill_between(np.arange(len(pred_x)), pred_x - pred_x_bottom_2std, pred_x + pred_x_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\angles_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(7)
# plt.xlabel('Nb of steps')
# plt.ylabel('v comparison between env and QRNN')
# plt.plot(env_v, label='env_next_states')
# plt.plot(pred_v, label='pred_next_states')
# plt.fill_between(np.arange(len(pred_v)), pred_v - pred_v_bottom_1std, pred_v + pred_v_top_1std, alpha=0.5, label='1 std', color='green')
# plt.fill_between(np.arange(len(pred_v)), pred_v - pred_v_bottom_2std, pred_v + pred_v_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\omegas_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(8)
# plt.xlabel('Nb of steps')
# plt.ylabel('theta comparison between env and QRNN')
# plt.plot(env_theta, label='env_next_states')
# plt.plot(pred_theta, label='pred_next_states')
# plt.fill_between(np.arange(len(pred_theta)), pred_theta - pred_theta_bottom_1std, pred_theta + pred_theta_top_2std, alpha=0.5, label='1 std', color='green')
# plt.fill_between(np.arange(len(pred_theta)), pred_theta - pred_theta_bottom_2std, pred_theta + pred_theta_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\angles_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(9)
# plt.xlabel('Nb of steps')
# plt.ylabel('omega comparison between env and QRNN')
# plt.plot(env_omega, label='env_next_states')
# plt.plot(pred_omega, label='pred_next_states')
# plt.fill_between(np.arange(len(pred_omega)), pred_omega - pred_omega_bottom_1std, pred_omega + pred_omega_top_1std, alpha=0.5, label='1 std', color='green')
# plt.fill_between(np.arange(len(pred_omega)), pred_omega - pred_omega_bottom_2std, pred_omega + pred_omega_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\omegas_comparison_GP_env_{timestamp}.png')
# plt.legend()




## Pendulum 

In [29]:
error_magnitudes_x = []
error_magnitudes_y = []
error_magnitudes_omega = []

env_x = []
env_y = []
env_omega = []

pred_x = []
pred_y = []
pred_omega = []

seed = 0
discrete = False
env = gym.make('Pendulum-v1')#.unwrapped
# sim_env = gym.make('Pendulum-v1')#.unwrapped  # Additional simulation model for MPC
# max_episode_steps = 500
# env = gym.make('CartPole-v1').unwrapped
# sim_env = gym.make('CartPole-v1').unwrapped  # Additional simulation model for MPC
sim_env.reset(seed=seed)
env.reset(seed=seed)

# Hyperparameters
state_dim = env.observation_space.shape[0]#-2#-1 # Since we only care about angle and omega which are given using env.state
# action_dim = env.action_space.shape[0]  # For Pendulum, it's continuous
action_dim=1

# Initialize the Next-State Prediction Network
model = NextStateMedianNetwork(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

loss_func = quantile_loss_median

states_low = torch.tensor([-1, -1, -8])
states_high = torch.tensor([1, 1, 8])

batch_size = 32

# Experience replay buffer
replay_buffer = []

num_test_steps = 20000

state, _ = env.reset(seed=seed)
episode_reward = 0
actions_list = []
done = False
truncated = False

actions_taken = np.zeros(num_test_steps)
env.action_space.seed(seed)

for step in range(num_test_steps):
    # state = env.state
    action = env.action_space.sample()
    actions_taken[step] = action
    
    # Apply the first action from the optimized sequence
    next_state, reward, done, truncated, info = env.step(action)
    # next_state = env.state
    episode_reward += reward
    actions_list.append(action)
    
    if step >= 1:
        # test_env = env.copy()
        # test_action = test_env.action_space.sample()
        # test_next_state, _, _, _, _ = test_env.step(test_action)
        # test_next_state = test_next_state['observation'][:3]
        
        test_next_state = torch.tensor(next_state, dtype=torch.float32)
        # test_next_state = torch.clip(test_next_state, states_low, states_high)
        # print("test_next_state ", test_next_state, "\n")
        
        if discrete:
            test_action = torch.tensor(np.array([action]), dtype=torch.float32)#.unsqueeze(1)  # Example action
        else:
            test_action = torch.tensor(action, dtype=torch.float32)#.unsqueeze(1)
        test_state = torch.tensor(state, dtype=torch.float32)#.reshape(1, -1)
        test_state = torch.clip(test_state, states_low, states_high)
        
        # Predict next state quantiles
        preds = model(test_state, test_action)  # Shape: (1, num_quantiles, state_dim)
        # lower_quantile = predicted_quantiles[:, 0, :]  # Shape: (1, state_dim)
        # mid_quantile = predicted_quantiles[:, int(num_quantiles/2), :].detach().numpy()  # Shape: (1, state_dim)
        # upper_quantile = predicted_quantiles[:, -1, :]  # Shape: (1, state_dim)
        # print("predicted_quantiles ", predicted_quantiles, "\n")
        # print("Lower Quantile:", lower_quantile, "\n")
        # print("Mid Quantile:", mid_quantile, "\n")
        # print("Upper Quantile:", upper_quantile, "\n")
        
        pred_x = np.append(pred_x, preds[0].detach().numpy())
        pred_y = np.append(pred_y, preds[1].detach().numpy())
        pred_omega = np.append(pred_omega, preds[2].detach().numpy())
        # # pred_theta2 = np.append(pred_sintheta2, preds[3].detach().numpy())  
        # pred_omega1 = np.append(pred_omega1, preds[4].detach().numpy())
        # pred_omega2 = np.append(pred_omega2, preds[5].detach().numpy())
        
        deltax = test_next_state[0] - preds[0]
        deltay = test_next_state[1] - preds[1]
        deltaomega = test_next_state[2] - preds[2]
        # deltaomega = test_next_state[3] - preds[3]
        # deltaomega1 = test_next_state[4] - preds[4]
        # deltaomega2 = test_next_state[5] - preds[5]

        # delta = env_test_state - mean_next_state.flatten()#.detach().numpy()
        # print("delta ", delta, "\n") # 6
        # print("delta_x ", deltax, "\n") # 7
        # error_magnitude = np.linalg.norm(delta)
        # error_magnitudes.append(error_magnitude)
        error_magnitudes_x.append(np.abs(deltax.detach().numpy())) # .detach().numpy()
        error_magnitudes_y.append(np.abs(deltay.detach().numpy())) # .detach().numpy()
        # error_magnitudes_theta.append(np.abs(deltatheta.detach().numpy())) # .detach().numpy()
        error_magnitudes_omega.append(np.abs(deltaomega.detach().numpy())) # .detach().numpy()
        # error_magnitudes_omega1.append(np.abs(deltaomega1.detach().numpy())) # .detach().numpy()
        # error_magnitudes_omega2.append(np.abs(deltaomega2.detach().numpy())) # .detach().numpy()
        # env_next_states.append(env_test_state)
        env_x.append(test_next_state[0])
        env_y.append(test_next_state[1])
        env_omega.append(test_next_state[2])
        # env_omega.append(test_next_state[3])
        # env_omega1.append(test_next_state[4])
        # env_omega2.append(test_next_state[5])
    
    # next_state = env.state.copy()
    # Store experience in replay buffer
    if discrete:
        replay_buffer.append((state, np.array([action]), reward, next_state, done))
    else:
        replay_buffer.append((state, np.array(action), reward, next_state, done))
    if len(replay_buffer) < batch_size:
        pass
    else:
        batch = random.sample(replay_buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.float32)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)
        
        states = torch.clip(states, states_low, states_high)
        
        # Predict next state quantiles
        predicted_quantiles = model(states, actions)  # Shape: (batch_size, num_quantiles, state_dim)
        
        # Use next state as target (can be improved with target policy)
        target_quantiles = next_states
        
        # Compute the target quantiles (e.g., replicate next state across the quantile dimension)
        # target_quantiles = next_states.unsqueeze(-1).repeat(1, 1, num_quantiles)

        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)

        loss = loss_func(predicted_quantiles, target_quantiles)
        
        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)
        
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    state = next_state
    
    if done or truncated:
        state, _ = env.reset(seed=seed)
        episode_reward = 0
        actions_list = []
        done = False
        truncated = False
 





In [30]:
print("Last 100 steps")
print(f"x comparison:",
np.mean(np.abs(np.array(pred_x[-100:])-np.array(env_x[-100:]))))

print(f"y comparison:",
np.mean(np.abs(np.array(pred_y[-100:])-np.array(env_y[-100:]))))

print(f"$\omega$ comparison:",
np.mean(np.abs(np.array(pred_omega[-100:])-np.array(env_omega[-100:]))))



Last 100 steps
x comparison: 0.008107431288808585
y comparison: 0.015759614994749427
$\omega$ comparison: 0.023014833331108094


In [31]:
print("All steps")
print(f"x comparison:",
np.mean(np.abs(np.array(pred_x)-np.array(env_x))))

print(f"y comparison:",
np.mean(np.abs(np.array(pred_y)-np.array(env_y))))

print(f"$\omega$ comparison:",
np.mean(np.abs(np.array(pred_omega)-np.array(env_omega))))



All steps
x comparison: 0.017279714541051934
y comparison: 0.018080271490379527
$\omega$ comparison: 0.03348231717623276


Plots for the pendulum env

In [32]:
# import matplotlib.pyplot as plt
# # # Plot the rewards
# # plt.figure(1)
# # # plt.legend()
# # # plt.grid()
# # plt.plot(episode_reward_list)
# # plt.xlabel('Nb of episodes')
# # plt.ylabel('episode reward')
# # # plt.title('MPC_Pendulum with Optimized Action Sequences')
# # # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\Graphs\\episode_rewards_withCPUforQRNN_{timestamp}.png')
# # # # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\episode_rewards_withGPUforGP_{timestamp}.png')

# plt.figure(1)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_x)
# plt.ylabel('error_magnitudes_x')
# plt.yscale('log')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_1_{timestamp}.png')

# plt.figure(2)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_y)
# plt.ylabel('error_magnitudes_y')
# plt.yscale('log')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_1_{timestamp}.png')

# plt.figure(3)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_omega)
# plt.ylabel('error_magnitudes_omega')
# plt.yscale('log')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_2_{timestamp}.png')


# plt.figure(4)
# plt.xlabel('Nb of steps')
# plt.ylabel('x comparison between env and QRNN')
# plt.plot(env_x, label='env_next_states')
# plt.plot(pred_x, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_theta)), pred_theta - pred_theta_bottom_1std, pred_theta + pred_theta_top_2std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_theta)), pred_theta - pred_theta_bottom_2std, pred_theta + pred_theta_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\angles_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(5)
# plt.xlabel('Nb of steps')
# plt.ylabel('y comparison between env and QRNN')
# plt.plot(env_y, label='env_next_states')
# plt.plot(pred_y, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_theta)), pred_theta - pred_theta_bottom_1std, pred_theta + pred_theta_top_2std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_theta)), pred_theta - pred_theta_bottom_2std, pred_theta + pred_theta_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\angles_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(6)
# plt.xlabel('Nb of steps')
# plt.ylabel('omega comparison between env and QRNN')
# plt.plot(env_omega, label='env_next_states')
# plt.plot(pred_omega, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_omega)), pred_omega - pred_omega_bottom_1std, pred_omega + pred_omega_top_1std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_omega)), pred_omega - pred_omega_bottom_2std, pred_omega + pred_omega_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\omegas_comparison_GP_env_{timestamp}.png')
# plt.legend()




## Mountain car

In [33]:
error_magnitudes_x = []
error_magnitudes_v = []

env_x = []
env_v = []

pred_x = []
pred_v = []

seed = 0
discrete = False
env = gym.make('MountainCarContinuous-v0')#.unwrapped
# sim_env = gym.make('Pendulum-v1')#.unwrapped  # Additional simulation model for MPC
# max_episode_steps = 500
# env = gym.make('CartPole-v1').unwrapped
# sim_env = gym.make('CartPole-v1').unwrapped  # Additional simulation model for MPC
sim_env.reset(seed=seed)
env.reset(seed=seed)

# Hyperparameters
state_dim = env.observation_space.shape[0]#-2#-1 # Since we only care about angle and omega which are given using env.state
# action_dim = env.action_space.shape[0]  # For Pendulum, it's continuous
action_dim=1

# Initialize the Next-State Prediction Network
model = NextStateMedianNetwork(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

loss_func = quantile_loss_median

states_low = torch.tensor([-1.2, -0.07])
states_high = torch.tensor([0.6, 0.07])

batch_size = 32

# Experience replay buffer
replay_buffer = []

num_test_steps = 20000

state, _ = env.reset(seed=seed)
episode_reward = 0
actions_list = []
done = False
truncated = False

actions_taken = np.zeros(num_test_steps)
env.action_space.seed(seed)

for step in range(num_test_steps):
    # state = env.state
    action = env.action_space.sample()
    actions_taken[step] = action
    
    # Apply the first action from the optimized sequence
    next_state, reward, done, truncated, info = env.step(action)
    # next_state = env.state
    episode_reward += reward
    actions_list.append(action)
    
    if step >= 1:
        # test_env = env.copy()
        # test_action = test_env.action_space.sample()
        # test_next_state, _, _, _, _ = test_env.step(test_action)
        # test_next_state = test_next_state['observation'][:3]
        
        test_next_state = torch.tensor(next_state, dtype=torch.float32)
        # test_next_state = torch.clip(test_next_state, states_low, states_high)
        # print("test_next_state ", test_next_state, "\n")
        
        if discrete:
            test_action = torch.tensor(np.array([action]), dtype=torch.float32)#.unsqueeze(1)  # Example action
        else:
            test_action = torch.tensor(action, dtype=torch.float32)#.unsqueeze(1)
        test_state = torch.tensor(state, dtype=torch.float32)#.reshape(1, -1)
        test_state = torch.clip(test_state, states_low, states_high)
        
        # Predict next state quantiles
        preds = model(test_state, test_action)  # Shape: (1, num_quantiles, state_dim)
        # lower_quantile = predicted_quantiles[:, 0, :]  # Shape: (1, state_dim)
        # mid_quantile = predicted_quantiles[:, int(num_quantiles/2), :].detach().numpy()  # Shape: (1, state_dim)
        # upper_quantile = predicted_quantiles[:, -1, :]  # Shape: (1, state_dim)
        # print("predicted_quantiles ", predicted_quantiles, "\n")
        # print("Lower Quantile:", lower_quantile, "\n")
        # print("Mid Quantile:", mid_quantile, "\n")
        # print("Upper Quantile:", upper_quantile, "\n")
        
        pred_x = np.append(pred_x, preds[0].detach().numpy())
        pred_v = np.append(pred_v, preds[1].detach().numpy())
        # pred_omega = np.append(pred_omega, preds[2].detach().numpy())
        # # pred_theta2 = np.append(pred_sintheta2, preds[3].detach().numpy())  
        # pred_omega1 = np.append(pred_omega1, preds[4].detach().numpy())
        # pred_omega2 = np.append(pred_omega2, preds[5].detach().numpy())
        
        deltax = test_next_state[0] - preds[0]
        deltav = test_next_state[1] - preds[1]
        # deltaomega = test_next_state[2] - preds[2]
        # deltaomega = test_next_state[3] - preds[3]
        # deltaomega1 = test_next_state[4] - preds[4]
        # deltaomega2 = test_next_state[5] - preds[5]

        # delta = env_test_state - mean_next_state.flatten()#.detach().numpy()
        # print("delta ", delta, "\n") # 6
        # print("delta_x ", deltax, "\n") # 7
        # error_magnitude = np.linalg.norm(delta)
        # error_magnitudes.append(error_magnitude)
        error_magnitudes_x.append(np.abs(deltax.detach().numpy())) # .detach().numpy()
        error_magnitudes_v.append(np.abs(deltav.detach().numpy())) # .detach().numpy()
        # error_magnitudes_theta.append(np.abs(deltatheta.detach().numpy())) # .detach().numpy()
        # error_magnitudes_omega.append(np.abs(deltaomega.detach().numpy())) # .detach().numpy()
        # error_magnitudes_omega1.append(np.abs(deltaomega1.detach().numpy())) # .detach().numpy()
        # error_magnitudes_omega2.append(np.abs(deltaomega2.detach().numpy())) # .detach().numpy()
        # env_next_states.append(env_test_state)
        env_x.append(test_next_state[0])
        env_v.append(test_next_state[1])
        # env_omega.append(test_next_state[2])
        # env_omega.append(test_next_state[3])
        # env_omega1.append(test_next_state[4])
        # env_omega2.append(test_next_state[5])
    
    # next_state = env.state.copy()
    # Store experience in replay buffer
    if discrete:
        replay_buffer.append((state, np.array([action]), reward, next_state, done))
    else:
        replay_buffer.append((state, np.array(action), reward, next_state, done))
    if len(replay_buffer) < batch_size:
        pass
    else:
        batch = random.sample(replay_buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.float32)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)
        
        states = torch.clip(states, states_low, states_high)
        
        # Predict next state quantiles
        predicted_quantiles = model(states, actions)  # Shape: (batch_size, num_quantiles, state_dim)
        
        # Use next state as target (can be improved with target policy)
        target_quantiles = next_states
        
        # Compute the target quantiles (e.g., replicate next state across the quantile dimension)
        # target_quantiles = next_states.unsqueeze(-1).repeat(1, 1, num_quantiles)

        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)

        loss = loss_func(predicted_quantiles, target_quantiles)
        
        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)
        
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    state = next_state
    
    if done or truncated:
        state, _ = env.reset(seed=seed)
        episode_reward = 0
        actions_list = []
        done = False
        truncated = False
 







In [34]:
print("Last 100 steps")
print(f"x comparison:",
np.mean(np.abs(np.array(pred_x[-100:])-np.array(env_x[-100:]))))

print(f"v comparison:",
np.mean(np.abs(np.array(pred_v[-100:])-np.array(env_v[-100:]))))



Last 100 steps
x comparison: 0.0034873494035855403
v comparison: 0.0008802143813227304


In [35]:
print("All steps")
print(f"x comparison:",
np.mean(np.abs(np.array(pred_x)-np.array(env_x))))

print(f"v comparison:",
np.mean(np.abs(np.array(pred_v)-np.array(env_v))))



All steps
x comparison: 0.004670445118318934
v comparison: 0.0019249889209457898


In [36]:
# import matplotlib.pyplot as plt
# # # Plot the rewards
# # plt.figure(1)
# # # plt.legend()
# # # plt.grid()
# # plt.plot(episode_reward_list)
# # plt.xlabel('Nb of episodes')
# # plt.ylabel('episode reward')
# # # plt.title('MPC_Pendulum with Optimized Action Sequences')
# # # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\Graphs\\episode_rewards_withCPUforQRNN_{timestamp}.png')
# # # # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\episode_rewards_withGPUforGP_{timestamp}.png')

# plt.figure(2)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_x)
# plt.ylabel('error_magnitudes_x')
# plt.yscale('log')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_1_{timestamp}.png')

# plt.figure(3)
# plt.xlabel('Nb of steps')
# plt.plot(error_magnitudes_v)
# plt.ylabel('error_magnitudes_v')
# plt.yscale('log')
# # plt.title('Error magnitude btw env.state and GP predicted state per step')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\error_magnitudes_2_{timestamp}.png')


# plt.figure(4)
# plt.xlabel('Nb of steps')
# plt.ylabel('x comparison between env and QRNN')
# plt.plot(env_x, label='env_next_states')
# plt.plot(pred_x, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_theta)), pred_theta - pred_theta_bottom_1std, pred_theta + pred_theta_top_2std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_theta)), pred_theta - pred_theta_bottom_2std, pred_theta + pred_theta_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\angles_comparison_GP_env_{timestamp}.png')
# plt.legend()

# plt.figure(5)
# plt.xlabel('Nb of steps')
# plt.ylabel('v comparison between env and QRNN')
# plt.plot(env_v, label='env_next_states')
# plt.plot(pred_v, label='pred_next_states')
# # plt.fill_between(np.arange(len(pred_omega)), pred_omega - pred_omega_bottom_1std, pred_omega + pred_omega_top_1std, alpha=0.5, label='1 std', color='green')
# # plt.fill_between(np.arange(len(pred_omega)), pred_omega - pred_omega_bottom_2std, pred_omega + pred_omega_top_2std, alpha=0.5, label='2 std', color='red')
# # plt.savefig(f'C:\\Users\\nicle\\Desktop\\GPBO-MBRL\\GP-MPC_NL\\PF_MPC_GP_Env\\ParallelOverParticles\\omegas_comparison_GP_env_{timestamp}.png')
# plt.legend()




## Panda Dense reward Reach (3D)

In [37]:
import panda_gym
error_magnitudes_x = []
error_magnitudes_y = []
error_magnitudes_z = []
error_magnitudes_vx = []
error_magnitudes_vy = []
error_magnitudes_vz = []

env_x = []
env_y = []
env_z = []
env_vx = []
env_vy = []
env_vz = []

pred_x = []
pred_y = []
pred_z = []
pred_vx = []
pred_vy = []
pred_vz = []


seed = 0
discrete = False
env = gym.make('PandaReach-v3')#.unwrapped # , render_mode='human'
# sim_env = gym.make('Pendulum-v1')#.unwrapped  # Additional simulation model for MPC
# max_episode_steps = 500
# env = gym.make('CartPole-v1').unwrapped
# sim_env = gym.make('CartPole-v1').unwrapped  # Additional simulation model for MPC
# sim_env.reset(seed=seed)
env.reset(seed=seed)

actions_low = env.action_space.low#[:3]
actions_high = env.action_space.high#[:3]
states_low = env.observation_space['observation'].low#[:3]
states_high = env.observation_space['observation'].high#[:3]
state_dim = len(states_low)
action_dim = len(actions_low)
states_low = torch.tensor([-10, -10, -10, -10, -10, -10])
states_high = torch.tensor([10, 10, 10, 10, 10, 10])
# states_low = env.observation_space['observation'].low#[:3]
# states_high = env.observation_space['observation'].high#[:3]


# Initialize the Next-State Prediction Network
model = NextStateMedianNetwork(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

loss_func = quantile_loss_median

batch_size = 32

# Experience replay buffer
replay_buffer = []

num_test_steps = 20000

state, _ = env.reset(seed=seed)
episode_reward = 0
actions_list = []
done = False
truncated = False

# actions_taken = np.zeros(num_test_steps)
actions_taken = np.zeros((num_test_steps, action_dim))
env.action_space.seed(seed)

goal_state = state['desired_goal'] # 3 components
state = state['observation']#[:3] # 6 components

for step in range(num_test_steps):
    # state = env.state
    action = env.action_space.sample()
    actions_taken[step] = action

    # Apply the first action from the optimized sequence
    next_state, reward, done, truncated, info = env.step(action)

    next_state = next_state['observation']#[:3] # env.state

    # next_state = env.state
    episode_reward += reward
    actions_list.append(action)
    
    if step >= 1:
        # test_env = env.copy()
        # test_action = test_env.action_space.sample()
        # test_next_state, _, _, _, _ = test_env.step(test_action)
        # test_next_state = test_next_state['observation'][:3]
        
        test_next_state = torch.tensor(next_state, dtype=torch.float32)
        # test_next_state = torch.clip(test_next_state, states_low, states_high)
        # print("test_next_state ", test_next_state, "\n")
        
        if discrete:
            test_action = torch.tensor(np.array([action]), dtype=torch.float32)#.unsqueeze(1)  # Example action
        else:
            test_action = torch.tensor(action, dtype=torch.float32)#.unsqueeze(1)
        test_state = torch.tensor(state, dtype=torch.float32)#.reshape(1, -1)
        test_state = torch.clip(test_state, states_low, states_high)
        
        # Predict next state quantiles
        preds = model(test_state, test_action)  # Shape: (1, num_quantiles, state_dim)
        # lower_quantile = predicted_quantiles[:, 0, :]  # Shape: (1, state_dim)
        # mid_quantile = predicted_quantiles[:, int(num_quantiles/2), :].detach().numpy()  # Shape: (1, state_dim)
        # upper_quantile = predicted_quantiles[:, -1, :]  # Shape: (1, state_dim)
        # print("predicted_quantiles ", predicted_quantiles, "\n")
        # print("Lower Quantile:", lower_quantile, "\n")
        # print("Mid Quantile:", mid_quantile, "\n")
        # print("Upper Quantile:", upper_quantile, "\n")
        
        pred_x = np.append(pred_x, preds[0].detach().numpy())
        pred_y = np.append(pred_y, preds[1].detach().numpy())
        pred_z = np.append(pred_z, preds[2].detach().numpy())
        pred_vx = np.append(pred_vx, preds[3].detach().numpy())  
        pred_vy = np.append(pred_vy, preds[4].detach().numpy())
        pred_vz = np.append(pred_vz, preds[5].detach().numpy())

        deltax = test_next_state[0] - preds[0]
        deltay = test_next_state[1] - preds[1]
        deltaz = test_next_state[2] - preds[2]
        deltavx = test_next_state[3] - preds[3]
        deltavy = test_next_state[4] - preds[4]
        deltavz = test_next_state[5] - preds[5]
        # delta = env_test_state - mean_next_state.flatten()#.detach().numpy()
        # print("delta ", delta, "\n") # 6
        # print("delta_x ", deltax, "\n") # 7
        # error_magnitude = np.linalg.norm(delta)
        # error_magnitudes.append(error_magnitude)
        error_magnitudes_x.append(np.abs(deltax.detach().numpy())) # .detach().numpy()
        error_magnitudes_y.append(np.abs(deltay.detach().numpy())) # .detach().numpy()
        error_magnitudes_z.append(np.abs(deltaz.detach().numpy())) # .detach().numpy()
        error_magnitudes_vx.append(np.abs(deltavx.detach().numpy())) # .detach().numpy()
        error_magnitudes_vy.append(np.abs(deltavy.detach().numpy())) # .detach().numpy()
        error_magnitudes_vz.append(np.abs(deltavz.detach().numpy())) # .detach().numpy()
        # error_magnitudes_xdist.append(np.abs(deltaxdist.detach().numpy())) # .detach().numpy()
        # error_magnitudes_ydist.append(np.abs(deltaydist.detach().numpy())) # .detach().numpy()
        # env_next_states.append(env_test_state)
        env_x.append(test_next_state[0])
        env_y.append(test_next_state[1])
        env_z.append(test_next_state[2])
        env_vx.append(test_next_state[3])
        env_vy.append(test_next_state[4])
        env_vz.append(test_next_state[5])
        # env_xdist.append(test_next_state[4])
        # env_ydist.append(test_next_state[5])
    
    # next_state = env.state.copy()
    # Store experience in replay buffer
    if discrete:
        replay_buffer.append((state, np.array([action]), reward, next_state, done))
    else:
        replay_buffer.append((state, np.array(action), reward, next_state, done))
    if len(replay_buffer) < batch_size:
        pass
    else:
        batch = random.sample(replay_buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.float32)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)
        
        states = torch.clip(states, states_low, states_high)
        
        # Predict next state quantiles
        predicted_quantiles = model(states, actions)  # Shape: (batch_size, num_quantiles, state_dim)
        
        # Use next state as target (can be improved with target policy)
        target_quantiles = next_states
        
        # Compute the target quantiles (e.g., replicate next state across the quantile dimension)
        # target_quantiles = next_states.unsqueeze(-1).repeat(1, 1, num_quantiles)

        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)

        loss = loss_func(predicted_quantiles, target_quantiles)
        
        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)
        
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    state = next_state
    
    if done or truncated:
        state, _ = env.reset(seed=seed)
        episode_reward = 0
        actions_list = []
        done = False
        truncated = False
        state = state['observation']#[:3] # 6 components
 







In [38]:
print("Last 100 steps")
print(f"x comparison:",
np.mean(np.abs(np.array(pred_x[-100:])-np.array(env_x[-100:]))))

print(f"y comparison:",
np.mean(np.abs(np.array(pred_y[-100:])-np.array(env_y[-100:]))))

print(f"z comparison:",
np.mean(np.abs(np.array(pred_z[-100:])-np.array(env_z[-100:]))))

print(f"vx comparison:",
np.mean(np.abs(np.array(pred_vx[-100:])-np.array(env_vx[-100:]))))

print(f"vy comparison:",
np.mean(np.abs(np.array(pred_vy[-100:])-np.array(env_vy[-100:]))))

print(f"vz comparison:",
np.mean(np.abs(np.array(pred_vz[-100:])-np.array(env_vz[-100:]))))





Last 100 steps
x comparison: 0.005005592107772827
y comparison: 0.004775416516058613
z comparison: 0.0035363725293427704
vx comparison: 0.07623940130637492
vy comparison: 0.044748367471038365
vz comparison: 0.06236906795296818


In [39]:
print("All steps")
print(f"x comparison:",
np.mean(np.abs(np.array(pred_x)-np.array(env_x))))

print(f"y comparison:",
np.mean(np.abs(np.array(pred_y)-np.array(env_y))))

print(f"z comparison:",
np.mean(np.abs(np.array(pred_z)-np.array(env_z))))

print(f"vx comparison:",
np.mean(np.abs(np.array(pred_vx)-np.array(env_vx))))

print(f"vy comparison:",
np.mean(np.abs(np.array(pred_vy)-np.array(env_vy))))

print(f"vz comparison:",
np.mean(np.abs(np.array(pred_vz)-np.array(env_vz))))




All steps
x comparison: 0.0067791285219022035
y comparison: 0.006416517839013276
z comparison: 0.005609462511235173
vx comparison: 0.0810434030699186
vy comparison: 0.06003168260204802
vz comparison: 0.08561910688359896


## MuJoCo Reacher (2D)


In [40]:
error_magnitudes_cos1 = []
error_magnitudes_cos2 = []
error_magnitudes_sin1 = []
error_magnitudes_sin2 = []
error_magnitudes_omega1 = []
error_magnitudes_omega2 = []
error_magnitudes_xdist = []
error_magnitudes_ydist = []

env_cos1 = []
env_cos2 = []
env_sin1 = []
env_sin2 = []
env_omega1 = []
env_omega2 = []
env_xdist = []
env_ydist = []

pred_cos1 = []
pred_cos2 = []
pred_sin1 = []
pred_sin2 = []
pred_omega1 = []
pred_omega2 = []
pred_xdist = []
pred_ydist = []


seed = 0
discrete = False
env = gym.make('Reacher-v5').unwrapped # , render_mode='human'
# sim_env = gym.make('Pendulum-v1')#.unwrapped  # Additional simulation model for MPC
# max_episode_steps = 500
# env = gym.make('CartPole-v1').unwrapped
# sim_env = gym.make('CartPole-v1').unwrapped  # Additional simulation model for MPC
# sim_env.reset(seed=seed)
env.reset(seed=seed)

actions_lows = env.action_space.low#[:3]
actions_highs = env.action_space.high#[:3]
states_low = torch.tensor([-1, -1, -1, -1, -100, -100, -torch.inf, -torch.inf])
states_high = torch.tensor([1, 1, 1, 1, 100, 100, torch.inf, torch.inf])
# states_low = env.observation_space['observation'].low#[:3]
# states_high = env.observation_space['observation'].high#[:3]
state_dim = 8
action_dim = len(actions_lows)

# Initialize the Next-State Prediction Network
model = NextStateMedianNetwork(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

loss_func = quantile_loss_median

batch_size = 32

# Experience replay buffer
replay_buffer = []

num_test_steps = 20000

state, _ = env.reset(seed=seed)
episode_reward = 0
actions_list = []
done = False
truncated = False

# actions_taken = np.zeros(num_test_steps)
actions_taken = np.zeros((num_test_steps, action_dim))
env.action_space.seed(seed)

goal_state = np.array([state[4], state[5]])
state = np.array([state[0], state[1], state[2], state[3], state[6], state[7], state[8], state[9]])

for step in range(num_test_steps):
    # state = env.state
    action = env.action_space.sample()
    actions_taken[step] = action

    # Apply the first action from the optimized sequence
    next_state, reward, done, truncated, info = env.step(action)

    goal_state = np.array([next_state[4], next_state[5]])
    next_state = np.array([next_state[0], next_state[1], next_state[2], next_state[3], next_state[6], next_state[7], next_state[8], next_state[9]])

    # next_state = env.state
    episode_reward += reward
    actions_list.append(action)
    
    if step >= 1:
        # test_env = env.copy()
        # test_action = test_env.action_space.sample()
        # test_next_state, _, _, _, _ = test_env.step(test_action)
        # test_next_state = test_next_state['observation'][:3]
        
        test_next_state = torch.tensor(next_state, dtype=torch.float32)
        # test_next_state = torch.clip(test_next_state, states_low, states_high)
        # print("test_next_state ", test_next_state, "\n")
        
        if discrete:
            test_action = torch.tensor(np.array([action]), dtype=torch.float32)#.unsqueeze(1)  # Example action
        else:
            test_action = torch.tensor(action, dtype=torch.float32)#.unsqueeze(1)
        test_state = torch.tensor(state, dtype=torch.float32)#.reshape(1, -1)
        test_state = torch.clip(test_state, states_low, states_high)
        
        # Predict next state quantiles
        preds = model(test_state, test_action)  # Shape: (1, num_quantiles, state_dim)
        # lower_quantile = predicted_quantiles[:, 0, :]  # Shape: (1, state_dim)
        # mid_quantile = predicted_quantiles[:, int(num_quantiles/2), :].detach().numpy()  # Shape: (1, state_dim)
        # upper_quantile = predicted_quantiles[:, -1, :]  # Shape: (1, state_dim)
        # print("predicted_quantiles ", predicted_quantiles, "\n")
        # print("Lower Quantile:", lower_quantile, "\n")
        # print("Mid Quantile:", mid_quantile, "\n")
        # print("Upper Quantile:", upper_quantile, "\n")
        
        pred_cos1 = np.append(pred_cos1, preds[0].detach().numpy())
        pred_cos2 = np.append(pred_cos2, preds[1].detach().numpy())
        pred_sin1 = np.append(pred_sin1, preds[2].detach().numpy())
        pred_sin2 = np.append(pred_sin2, preds[3].detach().numpy())  
        pred_omega1 = np.append(pred_omega1, preds[4].detach().numpy())
        pred_omega2 = np.append(pred_omega2, preds[5].detach().numpy())
        pred_xdist = np.append(pred_xdist, preds[6].detach().numpy())
        pred_ydist = np.append(pred_ydist, preds[7].detach().numpy())
        
        deltacos1 = test_next_state[0] - preds[0]
        deltacos2 = test_next_state[1] - preds[1]
        deltasin1 = test_next_state[2] - preds[2]
        deltasin2 = test_next_state[3] - preds[3]
        deltaomega1 = test_next_state[4] - preds[4]
        deltaomega2 = test_next_state[5] - preds[5]
        deltaxdist = test_next_state[6] - preds[6]
        deltaydist = test_next_state[7] - preds[7]

        # delta = env_test_state - mean_next_state.flatten()#.detach().numpy()
        # print("delta ", delta, "\n") # 6
        # print("delta_x ", deltax, "\n") # 7
        # error_magnitude = np.linalg.norm(delta)
        # error_magnitudes.append(error_magnitude)
        error_magnitudes_cos1.append(np.abs(deltacos1.detach().numpy())) # .detach().numpy()
        error_magnitudes_cos2.append(np.abs(deltacos2.detach().numpy())) # .detach().numpy()
        error_magnitudes_sin1.append(np.abs(deltasin1.detach().numpy())) # .detach().numpy()
        error_magnitudes_sin2.append(np.abs(deltasin2.detach().numpy())) # .detach().numpy()
        error_magnitudes_omega1.append(np.abs(deltaomega1.detach().numpy())) # .detach().numpy()
        error_magnitudes_omega2.append(np.abs(deltaomega2.detach().numpy())) # .detach().numpy()
        error_magnitudes_xdist.append(np.abs(deltaxdist.detach().numpy())) # .detach().numpy()
        error_magnitudes_ydist.append(np.abs(deltaydist.detach().numpy())) # .detach().numpy()
        # env_next_states.append(env_test_state)
        env_cos1.append(test_next_state[0])
        env_cos2.append(test_next_state[1])
        env_sin1.append(test_next_state[2])
        env_sin2.append(test_next_state[3])
        env_omega1.append(test_next_state[4])
        env_omega2.append(test_next_state[5])
        env_xdist.append(test_next_state[4])
        env_ydist.append(test_next_state[5])
    
    # next_state = env.state.copy()
    # Store experience in replay buffer
    if discrete:
        replay_buffer.append((state, np.array([action]), reward, next_state, done))
    else:
        replay_buffer.append((state, np.array(action), reward, next_state, done))
    if len(replay_buffer) < batch_size:
        pass
    else:
        batch = random.sample(replay_buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.float32)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)
        
        states = torch.clip(states, states_low, states_high)
        
        # Predict next state quantiles
        predicted_quantiles = model(states, actions)  # Shape: (batch_size, num_quantiles, state_dim)
        
        # Use next state as target (can be improved with target policy)
        target_quantiles = next_states
        
        # Compute the target quantiles (e.g., replicate next state across the quantile dimension)
        # target_quantiles = next_states.unsqueeze(-1).repeat(1, 1, num_quantiles)

        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)

        loss = loss_func(predicted_quantiles, target_quantiles)
        
        # # Compute Quantile Huber Loss
        # loss = quantile_huber_loss(predicted_quantiles, target_quantiles, quantiles)
        
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    state = next_state
    
    if done or truncated:
        state, _ = env.reset(seed=seed)
        episode_reward = 0
        actions_list = []
        done = False
        truncated = False
 







In [41]:
print("Last 100 steps")
print(f"cos1 comparison:",
np.mean(np.abs(np.array(pred_cos1[-100:])-np.array(env_cos1[-100:]))))

print(f"cos2 comparison:",
np.mean(np.abs(np.array(pred_cos2[-100:])-np.array(env_cos2[-100:]))))

print(f"sin1 comparison:",
np.mean(np.abs(np.array(pred_sin1[-100:])-np.array(env_sin1[-100:]))))

print(f"sin2 comparison:",
np.mean(np.abs(np.array(pred_sin2[-100:])-np.array(env_sin2[-100:]))))

print(f"omega1 comparison:",
np.mean(np.abs(np.array(pred_omega1[-100:])-np.array(env_omega2[-100:]))))

print(f"omega2 comparison:",
np.mean(np.abs(np.array(pred_omega1[-100:])-np.array(env_omega1[-100:]))))

print(f"xdist comparison:",
np.mean(np.abs(np.array(pred_xdist[-100:])-np.array(env_xdist[-100:]))))

print(f"ydist comparison:",
np.mean(np.abs(np.array(pred_ydist[-100:])-np.array(env_ydist[-100:]))))





Last 100 steps
cos1 comparison: 0.022095433305948972
cos2 comparison: 0.023208870827220382
sin1 comparison: 0.01948957162210718
sin2 comparison: 0.0208356861025095
omega1 comparison: 6.457271306742914
omega2 comparison: 0.04880671992897987
xdist comparison: 6.046130554331467
ydist comparison: 5.331882708300836


In [42]:
print("All steps")
print(f"cos1 comparison:",
np.mean(np.abs(np.array(pred_cos1)-np.array(env_cos1))))

print(f"cos2 comparison:",
np.mean(np.abs(np.array(pred_cos2)-np.array(env_cos2))))

print(f"sin1 comparison:",
np.mean(np.abs(np.array(pred_sin1)-np.array(env_sin1))))

print(f"sin2 comparison:",
np.mean(np.abs(np.array(pred_sin2)-np.array(env_sin2))))

print(f"omega1 comparison:",
np.mean(np.abs(np.array(pred_omega1)-np.array(env_omega2))))

print(f"omega2 comparison:",
np.mean(np.abs(np.array(pred_omega1)-np.array(env_omega1))))

print(f"xdist comparison:",
np.mean(np.abs(np.array(pred_xdist)-np.array(env_xdist))))

print(f"ydist comparison:",
np.mean(np.abs(np.array(pred_ydist)-np.array(env_ydist))))




All steps
cos1 comparison: 0.049327370363366305
cos2 comparison: 0.04423726345264709
sin1 comparison: 0.04861820125431189
sin2 comparison: 0.04721734981839718
omega1 comparison: 10.88703780446113
omega2 comparison: 0.11799898348705727
xdist comparison: 9.126787669140551
ydist comparison: 5.619604415049504
